In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import torch 
from argparse import Namespace
from tqdm import tqdm
import pickle 
import glob 
import ast

# Add the path to the directory containing the sybil module
sys.path.append('/workspace/home/tengyuezhang/sybil_cect/code/Sybil/')
from sybil.utils.metrics import concordance_index, get_survival_metrics
from sybil import Sybil, Serie
from sybil import visualize_attentions


In [30]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
num_threads = os.cpu_count() // 5

In [31]:
ucla_ldct_cases_path = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_ldct/ucla_ldct_final_20_cases_outcome.csv'
output_feature_path = '/workspace/home/tengyuezhang/sybil_cect/results/ucla_ldct/ucla_ldct_20_raw_features.csv'

In [32]:
# Initialize the Sybil model
model = Sybil("sybil_1")
num_years = 6

/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Running one example

In [ ]:
# Load the CSV file
all_cases = pd.read_csv(ucla_ldct_cases_path)
example = all_cases.iloc[1]
example

In [ ]:
dicom_dir = example['Directory']
event = 0
years_to_event = 1
pid = example['pid']
dicom_list = glob.glob(dicom_dir + '/*')
serie = Serie(dicom_list, label=event, censor_time=years_to_event)

results = model.predict([serie], return_attentions=True, threads=num_threads)

In [ ]:
results 

In [ ]:
results.attentions[0]['features'][0, 0, :].tolist()

## On the entire dataset

In [33]:
# Load the CSV file
all_cases = pd.read_csv(ucla_ldct_cases_path)
df = all_cases # initialize the output df with the input csv file 
num_features = 512 

In [34]:
for i in range(num_years):
    df[f'pred_risk_year_{i}'] = np.nan
for i in range(num_features):
    df[f'feature_{i}'] = np.nan 

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
    dicom_dir = row['Directory']
    event = 0
    years_to_event = 1
    pid = row['pid']
    dicom_list = glob.glob(dicom_dir + '/*')
    serie = Serie(dicom_list, label=event, censor_time=years_to_event)
    
    # get predicted risk scores and features from the last hidden layer (returned along with the attentions)
    results = model.predict([serie], return_attentions=True, threads=num_threads)
        
    # append risk scores 
    for i in range(num_years):
        df.at[index, f'pred_risk_year_{i}'] = results.scores[0][i]
    
    # append features 
    for i in range(num_features):
        # 'feature' for before relu 
        #'hidden' for after relu
        df.at[index, f'feature_{i}'] = results.attentions[0]['features'][0, 0, i] 
        
    # save updated df 
    df.to_csv(output_feature_path, index=False)

/tmp/ipykernel_1583/775224613.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'feature_{i}'] = np.nan
/tmp/ipykernel_1583/775224613.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'feature_{i}'] = np.nan
/tmp/ipykernel_1583/775224613.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 

Processing cases:   0%|                                                                                           | 0/20 [00:00<?, ?it/s]

using device cuda:0


Processing cases:   5%|████▏                                                                              | 1/20 [00:20<06:35, 20.82s/it]

using device cuda:0


...

Processing cases:  90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [05:33<00:35, 17.66s/it]

using device cuda:0


Processing cases:  95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [05:57<00:19, 19.48s/it]

using device cuda:0


Processing cases: 100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [06:17<00:00, 18.87s/it]


# Append diagnosis column 

In [ ]:
# read excel spreadsheet from path ucla_ldct_scores 
# read the diagnosis column and the pid column 
# match the pids. assign diagnosis to our df based on the info from the excel 

In [ ]:
# !pip install openpyxl

In [ ]:
# excel_file = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_ldct/ucla_ldct_risk_scores.xlsx'
# excel_df = pd.read_excel(excel_file)
# excel_df = excel_df[["pid", "diagnosis"]] 
# excel_df

In [ ]:
# excel_df = excel_df.dropna(subset=['diagnosis'])
# excel_df

In [ ]:
# merged_df = df.merge(excel_df, on="pid", how="left")
# merged_df

In [ ]:
# len(merged_df)

In [ ]:
# final_case_select_with_outcome_path = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_ldct/ucla_ldct_final_20_cases_outcome.csv'
# merged_df.to_csv(final_case_select_with_outcome_path, index=False)